In [1]:
NUM_CLASSES = 43
IMAGE_SIZE = 96
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
from keras import backend as K
K.set_image_data_format('channels_first')
#K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
import glob, os
import numpy as np
import pandas as pd
from skimage import io, color, exposure, transform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.utils import np_utils
from keras import backend as K, applications, Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

In [3]:
def preprocess_img(img):
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]

    #img = color.rgb2gray(img)
    img = transform.resize(img, (IMAGE_SIZE, IMAGE_SIZE))

    img = np.rollaxis(img,-1)

    return img

In [4]:
def load_train_dataset(root_dir):
    imgs = []
    labels = []
   
    all_img_paths = glob.glob(root_dir+'*/*.ppm', recursive=True)
    np.random.shuffle(all_img_paths)
    for img_path in all_img_paths:
        try:
            img = preprocess_img(io.imread(img_path))
            label = int(img_path.split('\\')[-2])
            imgs.append(img)
            labels.append(label)

            #if len(imgs) % 1000 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
        except (IOError, OSError):
            print('missed', img_path)
            pass

    X = np.array(imgs, dtype='float32')
    Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    return X, Y

In [5]:
def load_test_datatset(gt_path, test_dataset_path):
    gt = pd.read_csv(gt_path, sep=';')
    X_test = []
    Y_test = []
    for file_name, class_id in zip(list(gt['Filename']), list(gt['ClassId'])):
        img_path = os.path.join(test_dataset_path, file_name)
        X_test.append(preprocess_img(io.imread(img_path)))
        Y_test.append(class_id)

    return np.array(X_test), np.array(Y_test)


In [9]:
X_train_orig, Y_train_orig = load_train_dataset('GTSRB\Final_Training\Images\\')

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [10]:
X_test_orig, Y_test_orig = load_test_datatset('GT-final_test.csv', 'GTSRB\Final_Test\Images\\')

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [29]:
print(X_test_orig.shape)
print(Y_test_orig.shape)
print(X_train_orig.shape)
print(Y_train_orig.shape)

(12630, 3, 96, 96)
(12630,)
(39209, 3, 96, 96)
(39209, 43)


In [31]:
Y_test = np_utils.to_categorical(Y_test_orig, NUM_CLASSES)
#Y_test.shape
X_train = X_train_orig#.reshape(X_train_orig.shape[0], 1, IMAGE_SIZE, IMAGE_SIZE)
Y_train = Y_train_orig #np_utils.to_categorical(Y_train_orig, NUM_CLASSES)

X_test = X_test_orig#.reshape(X_test_orig.shape[0], 1, IMAGE_SIZE, IMAGE_SIZE)

In [13]:
from keras.layers.core import  Lambda
from keras import backend as K

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.merge import concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2

# Paper: https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf
# Caffe code: https://github.com/BVLC/caffe/tree/master/models/bvlc_alexnet
# Adapted from: https://github.com/heuritech/convnets-keras

def crosschannelnormalization(alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
    """
    This is the function used for cross channel normalization in the original
    Alexnet
    """


    def f(X):
        if K.image_dim_ordering()=='tf':
            b, r, c, ch = X.get_shape()
        else:
            b, ch, r, c = X.shape

        half = n // 2
        square = K.square(X)
        scale = k
        if K.image_dim_ordering() == 'th':
            extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0, 2, 3, 1)), ((0, 0),(half,half)))
            extra_channels = K.permute_dimensions(extra_channels, (0, 3, 1, 2))
            for i in range(n):
                scale += alpha * extra_channels[:, i:i+ch, :, :]
        if K.image_dim_ordering() == 'tf':
            extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0, 3, 1, 2)), ((half,half),(0, 0)))
            extra_channels = K.permute_dimensions(extra_channels, (0, 2, 3, 1))
            for i in range(n):
                scale += alpha * extra_channels[:, :, :, i:i+int(ch)]
        scale = scale ** beta
        return X / scale

    return Lambda(f, output_shape=lambda input_shape: input_shape, **kwargs)


def splittensor(axis=1, ratio_split=1, id_split=0, **kwargs):
    def f(X):
        div = K.shape(X)[axis] // ratio_split

        if axis == 0:
            output = X[id_split*div:(id_split+1)*div, :, :, :]
        elif axis == 1:
            output = X[:, id_split*div:(id_split+1)*div, :, :]
        elif axis == 2:
            output = X[:, :, id_split*div:(id_split+1)*div, :]
        elif axis == 3:
            output = X[:, :, :, id_split*div:(id_split+1)*div]
        else:
            raise ValueError("This axis is not possible")

        return output

    def g(input_shape):
        output_shape = list(input_shape)
        output_shape[axis] = output_shape[axis] // ratio_split
        return tuple(output_shape)

    return Lambda(f, output_shape=lambda input_shape: g(input_shape), **kwargs)


def build_alexNet(img_shape=(3, 227, 227), n_classes=1000, l2_reg=0.):

    dim_ordering = K.image_dim_ordering()
    if dim_ordering == 'th':
        batch_index = 0
        channel_index = 1
        row_index = 2
        col_index = 3
    if dim_ordering == 'tf':
        batch_index = 0
        channel_index = 3
        row_index = 1
        col_index = 2

    inputs = Input(img_shape)

    conv_1 = Convolution2D(96, 11, 11, subsample=(4, 4), activation='relu',
                           name='conv_1', W_regularizer=l2(l2_reg))(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2, 2))(conv_2)
    conv_2 = concatenate([
        Convolution2D(128, 5, 5, activation="relu", name='conv_2_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_2)
        ) for i in range(2)], axis=channel_index, name="conv_2")

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1, 1))(conv_3)
    conv_3 = Convolution2D(384, 3, 3, activation='relu', name='conv_3',
                           W_regularizer=l2(l2_reg))(conv_3)

    conv_4 = ZeroPadding2D((1, 1))(conv_3)
    conv_4 = concatenate([
        Convolution2D(192, 3, 3, activation="relu", name='conv_4_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_4)
        ) for i in range(2)], axis=channel_index, name="conv_4")

    conv_5 = ZeroPadding2D((1, 1))(conv_4)
    conv_5 = concatenate([
        Convolution2D(128, 3, 3, activation="relu", name='conv_5_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_5)
        ) for i in range(2)], axis=channel_index, name="conv_5")

    dense_1 = MaxPooling2D((3, 3), strides=(2, 2), name="convpool_5")(conv_5)

    dense_1 = Flatten(name="flatten")(dense_1)
    dense_1 = Dense(4096, activation='relu', name='dense_1',
                    W_regularizer=l2(l2_reg))(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu', name='dense_2',
                    W_regularizer=l2(l2_reg))(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    if n_classes == 1000:
        dense_3 = Dense(n_classes, name='dense_3',
                        W_regularizer=l2(l2_reg))(dense_3)
    else:
        # We change the name so when loading the weights_file from a
        # Imagenet pretrained model does not crash
        dense_3 = Dense(n_classes, name='dense_3_new',
                        W_regularizer=l2(l2_reg))(dense_3)
    prediction = Activation("softmax", name="softmax")(dense_3)

    model = Model(input=inputs, output=prediction)

    return model

In [38]:
model = build_alexNet((3, IMAGE_SIZE, IMAGE_SIZE))
model.summary()

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv_1", strides=(4, 4), kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_1", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_2", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(384, (3, 3), activation="relu", name="conv_3", kernel_regularizer=<keras.reg...)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 22, 22)   34944       input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 96, 10, 10)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
convpool_1 (Lambda)             (None, 96, 10, 10)   0           max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
zero_paddi

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv_5_1", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:117: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv_5_2", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:123: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4096, activation="relu", name="dense_1", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:126: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4096, activation="relu", name="dense_2", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Lo

In [41]:
#model.load_weights("alexnet_weights.h5")
x = model.output
#x = Flatten()(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
# creating the final model
model_final = Model(model.input, output=predictions)

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("de...)`
  import sys


In [42]:
model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model_final.fit(X_train, Y_train,
          epochs=20,
          batch_size=128,
          shuffle=True,
          verbose=1,
          validation_data=(X_test, Y_test))




Train on 39209 samples, validate on 12630 samples
Epoch 1/20
39209/39209 [==============================] - 593s 15ms/step - loss: 3.6473 - acc: 0.0545 - val_loss: 3.5567 - val_acc: 0.0570
Epoch 2/20
39209/39209 [==============================] - 565s 14ms/step - loss: 3.5542 - acc: 0.0566 - val_loss: 3.5010 - val_acc: 0.0594
Epoch 3/20
39209/39209 [==============================] - 566s 14ms/step - loss: 3.5272 - acc: 0.0536 - val_loss: 3.4821 - val_acc: 0.0594
Epoch 4/20
39209/39209 [==============================] - 565s 14ms/step - loss: 3.5141 - acc: 0.0565 - val_loss: 3.4740 - val_acc: 0.0570
Epoch 5/20
39209/39209 [==============================] - 568s 14ms/step - loss: 3.5044 - acc: 0.0570 - val_loss: 3.4699 - val_acc: 0.0594
Epoch 6/20
39209/39209 [==============================] - 567s 14ms/step - loss: 3.5004 - acc: 0.0565 - val_loss: 3.4677 - val_acc: 0.0594
Epoch 7/20
39209/39209 [==============================] - 567s 14ms/step - loss: 3.4961 - acc: 0.0568 - val_loss: 3.

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [44]:
score = model_final.evaluate(X_test, Y_test, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

model_final.summary()

12630/12630 [==============================] - 74s 6ms/step
Test score: 3.463088956109513
Test accuracy: 0.05938242280285035
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 22, 22)   34944       input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 96, 10, 10)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
convpool_1 (Lambda)             (None, 96, 10, 10)   0           max_pooling2d_5[0]

In [47]:
model_pre_trained = build_alexNet()
model_pre_trained.load_weights("alexnet_weights.h5")
model_pre_trained.summary()

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv_1", strides=(4, 4), kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_1", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_2", kernel_regularizer=<keras.reg...)`
C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(384, (3, 3), activation="relu", name="conv_3", kernel_regularizer=<keras.reg...)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 3, 227, 227)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 55, 55)   34944       input_13[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 96, 27, 27)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
convpool_1 (Lambda)             (None, 96, 27, 27)   0           max_pooling2d_11[0][0]           
__________________________________________________________________________________________________
zero_paddi

In [48]:
newInput = Input((3, IMAGE_SIZE, IMAGE_SIZE))    # let us say this new InputLayer
newOutputs = model_pre_trained(newInput)
newModel = Model(newInput, newOutputs)

newModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 3, 96, 96)         0         
_________________________________________________________________
model_17 (Model)             (None, 1000)              60965224  
Total params: 60,965,224
Trainable params: 60,965,224
Non-trainable params: 0
_________________________________________________________________


In [50]:
for layer in newModel.layers:
    layer.trainable = False
x = newModel.output
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# creating the final model
model__pre_trained_final = Model(input=newModel.input, output=predictions)

C:\Users\stsh0318\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [51]:
model__pre_trained_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
score = model__pre_trained_final.evaluate(X_test, Y_test, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

model__pre_trained_final.summary()

InvalidArgumentError: Matrix size-incompatible: In[0]: [32,256], In[1]: [9216,4096]
	 [[{{node model_17/dense_1/MatMul}}]]